In [30]:
"""Modified from https://github.com/mlfoundations/open_clip"""

from typing import Optional, Tuple

import collections
import logging
import os
import numpy as np
import pandas as pd
import torch
import torchvision.datasets as datasets
from torchvision import transforms as T
from PIL import Image, ImageFont
from torch.utils.data import Dataset
import json
# from fromage import utils
import torch.nn.functional as F
import pickle
import pdb




def get_dataset(args,split, tokenizer, precision: str = 'fp32') -> Dataset:
    dataset= CsvDataset(tokenizer, '/home/ubuntu/Project/11-777-Project-aa/Data/WebQA_train_val_final.json',split=split)
    return dataset


class CsvDataset(Dataset):
  def __init__(self, tokenizer,dataset_json_path,use_num_samples=-1,
               split=['val'],Qcate=['text'], max_len: int = 32, 
               sep="\t", precision: str = 'fp32'):
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.precision = precision

    assert os.path.exists(dataset_json_path), "loader.Dataset: dataset json file doesn't exist! {}".format(dataset_json_path)
    # self.imgid_map=pickle.load(open('/home/ubuntu/Project/11-777-Project-aa/Data/image_id_map_0328.pkl', "rb"))

    self.instance_list = []
    count = 0
    with open(dataset_json_path, "r") as f:
        dataset_J = json.load(f)
        for i in dataset_J:
            datum = dataset_J[i]
            # if datum['Qcate'] == 'txt':
            if use_num_samples == -1 or count < use_num_samples:
                guid = datum['Guid']
                qcate = datum['Qcate'] if 'Qcate' in datum else 'TBD'
                Q = datum['Q'].replace('"', "")
                A = datum['A'][0].replace('"', "")
                A_list = [a.replace('"', "") for a in datum['A']]
                try: Keywords_A = datum['Keywords_A'].replace('"', "")
                except: Keywords_A = "TBD"
                gold_facts = []
                

                for idx,fa in enumerate(datum['txt_posFacts_path']):
                    # encode fa using a encoder function and save the path by creating a new key 'txt_posFacts_path'
                    print('1')
                    fa=fa.replace('/Users/aavi/Desktop/11-777-Project/Data/univlr_features','/home/ubuntu/Project/11-777-Project-aa/Data/univlr_features')
                    gold_facts.append(fa)
                self.instance_list.append((gold_facts, [], [], [], Q, A, Keywords_A, A_list, False, "txt", guid, qcate)) # do_filter_task, context
                
                count += 1
            else: 
                  # if ('all' in Qcate) or datum['Qcate'] in Qcate:
                  #     if use_num_samples == -1 or count < use_num_samples:
                  #         guid = datum['Guid']
                  #         qcate = datum['Qcate'] if 'Qcate' in datum else 'TBD'
                  #         Q = datum['Q'].replace('"', "")
                  #         A = datum['A'][0].replace('"', "")
                  #         A_list = [a.replace('"', "") for a in datum['A']]
                  #         try: Keywords_A = datum['Keywords_A'].replace('"', "")
                  #         except: Keywords_A = "TBD"
                  #         gold_feature = []
            
                  #         for idx,fa in enumerate(datum['img_posFacts_path']):
                  #         # encode fa using a encoder function and save the path by creating a new key 'txt_posFacts_path'
                  #           fa=fa.replace('/Users/aavi/Desktop/11-777-Project/Data/univlr_features','/home/ubuntu/Project/11-777-Project-aa/Data/univlr_features')
                  #           gold_feature.append(fa)
                  #         self.instance_list.append((gold_feature, [], [], [], Q, A, Keywords_A, A_list, False, "img", guid, qcate)) # do_filter_task, context )
                  #         count += 1
                  pass
        print("Load {} instances from {} samples".format(len(self.instance_list), count))



    logging.debug('Done loading data.')

  def __len__(self):
    return len(self.instance_list)

  def __getitem__(self, idx):
    gold_feature, [], [], [], Q, A, Keywords_A, A_list, _, type1, _, qcate= self.instance_list[idx]
    while(len(gold_feature)<2):
      gold_feature.append(gold_feature[0])
    tokenized_data = self.tokenizer(
        A,
        return_tensors="pt",
        padding='max_length',
        truncation=True,
        max_length=self.max_len)
    tokens = tokenized_data.input_ids[0]
    Q='Q: ' + Q + '\nA:'
    caption_len = tokenized_data.attention_mask[0].sum()

      
    return gold_feature, Q, tokens, caption_len,type1,A
  

In [31]:
def collate_fn(batch):
    # Transpose the batch (convert a list of tuples to a tuple of lists)
    batch = list(zip(*batch))

    # Process each component separately
    visual_features = batch[0]  # Assuming the first element is visual features
    questions = batch[1]  # Assuming the second element is questions
    tokens = torch.stack(batch[2])  # Assuming the third element is tokens
    caption_lengths = torch.tensor(batch[3])  # Assuming the fourth element is caption lengths
    types = batch[4]  # Assuming the fifth element is data types ("img" or "txt")
    answers = batch[5]
    return visual_features, questions, tokens, caption_lengths, types, answers

In [32]:
import torch

from transformers import AutoTokenizer
opt_version= 'facebook/opt-1.3b'
# Data loading code
tokenizer = AutoTokenizer.from_pretrained(opt_version, use_fast=False)
train_dataset = get_dataset( 'd',['train'], tokenizer)
# val_dataset = get_dataset( 'd',['val'], tokenizer)
# print(f'Training with {len(train_dataset)} examples and validating with {len(val_dataset)} examples.')

train_sampler = None
val_sampler = None

train_loader = torch.utils.data.DataLoader(
train_dataset, batch_size=4,
num_workers=4,collate_fn=collate_fn)
# val_loader = torch.utils.data.DataLoader(
# val_dataset, batch_size=2, shuffle=False,
# num_workers=4)


KeyError: 'txt_posFacts_path'

In [11]:
#generate samples for training
for batch_idx, (visual_features, questions, tokens, caption_lengths, types,A) in enumerate(train_loader):
    print(visual_features)
    print(questions)
    print(tokens)
    print(caption_lengths.shape)
    print(types)
    print(A)
    break


(['/home/ubuntu/Project/11-777-Project-aa/Data/univlr_features/d5bbc6d80dba11ecb1e81171463288e9_0.pkl', '/home/ubuntu/Project/11-777-Project-aa/Data/univlr_features/d5bbc6d80dba11ecb1e81171463288e9_1.pkl'], ['/home/ubuntu/Project/11-777-Project-aa/Data/univlr_features/d5bbc7640dba11ecb1e81171463288e9_0.pkl', '/home/ubuntu/Project/11-777-Project-aa/Data/univlr_features/d5bbc7640dba11ecb1e81171463288e9_1.pkl'], ['/home/ubuntu/Project/11-777-Project-aa/Data/univlr_features/d5bbc8cc0dba11ecb1e81171463288e9_0.pkl', '/home/ubuntu/Project/11-777-Project-aa/Data/univlr_features/d5bbc8cc0dba11ecb1e81171463288e9_1.pkl'], ['/home/ubuntu/Project/11-777-Project-aa/Data/univlr_features/d5bbc91c0dba11ecb1e81171463288e9_0.pkl', '/home/ubuntu/Project/11-777-Project-aa/Data/univlr_features/d5bbc91c0dba11ecb1e81171463288e9_1.pkl'])
('Q: Are both the National Museum of the American Indian in Washington, D.C. and the Xanadu House in Kissimmee, Florida the same color?\nA:', 'Q: Are the satellites on the Sov

In [ ]:
# print the next batch
visual_features, questions, tokens, caption_lengths, types=next(iter(train_loader))
# print(len(visual_features[0]))
# print(len(questions))
# print(tokens.shape)
# print(caption_lengths.shape)
# print(types)

In [62]:
len(visual_features)

2

In [33]:
import torch
from torchvision import io

def read_and_stack_features(batch_paths):
    features_list = []

    for path_pair in batch_paths:
        feature1_path, feature2_path = path_pair

        # Read features
        with open(feature1_path, "rb") as f:
            feature1 = pickle.load(f)
        feature1 = feature1['fc1_features']
        with open(feature2_path, "rb") as f:
            feature2 = pickle.load(f)
        feature2 = feature2['fc1_features']
        print(feature1.shape)
        # Assuming feature1 and feature2 are PyTorch tensors
        # If not, you may need to convert them to tensors using torch.tensor()

        # Stack features along a new dimension (concatenate along the last axis)
        #stacked_features = torch.stack([feature1, feature2])
        merged_feature = torch.cat((feature1, feature2), 1)

        # Append the stacked features to the list
        features_list.append(merged_feature)

    # Stack the list of stacked features along a new dimension
    batch_tensor = torch.stack(features_list)

    return batch_tensor

# Example usage:
batch_paths = [
    ('path/to/feature1/image1.png', 'path/to/feature2/image1.png'),
    ('path/to/feature1/image2.png', 'path/to/feature2/image2.png'),
    # Add more paths as needed
]

result = read_and_stack_features(visual_features)
print(result.shape)

torch.Size([100, 2048])
torch.Size([100, 2048])
torch.Size([2, 100, 4096])


In [1]:
import torch
import torch.nn as nn

# Input tensor with size (2, 100, 4096)
input_tensor = torch.randn((2, 100, 4096))

# Define a model with linear layers and reshaping
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.linear_layer = nn.Linear(4096, 2048)

    def forward(self, x):
        # Apply linear layer
        x = self.linear_layer(x)

        # Reshape the tensor to (2, 4, 2048)
        x = x.view(x.size(0), 4, -1)

        return x

# Instantiate the model
model = CustomModel()

# Forward pass
output_tensor = model(input_tensor)

# Print the size of the output tensor
print(output_tensor.size())

torch.Size([2, 4, 51200])


In [2]:
409600/(2048*4)

50.0

In [8]:
import torch
import torch.nn as nn

# Assuming you have input data x with size (2, 100, 4096)
x = torch.rand((2, 100, 4096))

# Define a simple neural network model
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()

        # Linear layer 1: Reduce the dimension from 4096 to 2048
        self.linear1 = nn.Linear(100, 2)




    def forward(self, x):
        x=x.view(x.size(0),-1,x.size(1))
        x = self.linear1(x)
        x = x.reshape(x.size(0), 4, -1)
       
        return x

# Create an instance of the model
model = CustomModel()

# Forward pass to transform the input tensor
output = model(x)

# Print the size of the output tensor
print(output.size())


torch.Size([2, 4, 2048])
